In [15]:
import torch
from scipy.io import wavfile
import wavio
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import numpy as np
import soundfile

In [16]:
waveform_original,sample_rate_original= soundfile.read('data\\2_sets\\A_\\'+'full_A.wav')
waveform_1,_  = soundfile.read('data\\2_sets\\A_\\'+'full_A_d1.wav')
waveform_2,_ = soundfile.read('data\\2_sets\\A_\\'+'full_A_d2.wav')
waveform_3,_ = soundfile.read('data\\2_sets\\A_\\'+'full_A_d3.wav')
waveform_4,_ = soundfile.read('data\\2_sets\\A_\\'+'full_A_n1.wav')
waveform_5,_ = soundfile.read('data\\2_sets\\A_\\'+'full_A_n2.wav')
test_audio,_ = soundfile.read('data\\2_sets\\A\\'+'full_B_d1.wav') 
waveform_original = waveform_original.astype('float32')
waveform_original /= 255
print("Shape of waveform: {}".format(np.shape(waveform_original)))
print("Sample rate of waveform: {}".format(sample_rate_original))
print(np.shape(waveform_original))



Shape of waveform: (37847250, 2)
Sample rate of waveform: 44100
(37847250, 2)


In [17]:
samples=20
BATCH_SIZE = 10
waveform_1 = waveform_3.astype('float32')
waveform_1 /= 255
waveform_2 = waveform_3.astype('float32')
waveform_2 /= 255
waveform_3 = waveform_3.astype('float32')
waveform_3 /= 255
waveform_4 = waveform_3.astype('float32')
waveform_4 /= 255
waveform_5 = waveform_3.astype('float32')
waveform_5 /= 255
test_audio = test_audio.astype('float32')
test_audio /=255

waveform_original = np.repeat(torch.tensor(np.transpose(waveform_original))[np.newaxis,...], BATCH_SIZE, axis=0)
y1 = np.repeat(torch.tensor(np.transpose(waveform_1))[np.newaxis,...], samples, axis=0)
y2 = np.repeat(torch.tensor(np.transpose(waveform_2))[np.newaxis,...], samples, axis=0)
y3 = np.repeat(torch.tensor(np.transpose(waveform_3))[np.newaxis,...], samples, axis=0)
y4 = np.repeat(torch.tensor(np.transpose(waveform_4))[np.newaxis,...], samples, axis=0)
y5 = np.repeat(torch.tensor(np.transpose(waveform_5))[np.newaxis,...], samples, axis=0)

y = np.concatenate((y1,y2,y3,y4,y5),axis=0)
print(np.shape(waveform_original))


np.random.shuffle(y)



print(np.shape(y3))
print(y3)

torch.Size([10, 2, 37847250])
torch.Size([20, 2, 37847250])
tensor([[[ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.4361e-06,
          -7.1806e-07, -1.3164e-06],
         [ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.1968e-06,
          -9.5741e-07, -1.0771e-06]],

        [[ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.4361e-06,
          -7.1806e-07, -1.3164e-06],
         [ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.1968e-06,
          -9.5741e-07, -1.0771e-06]],

        [[ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.4361e-06,
          -7.1806e-07, -1.3164e-06],
         [ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.1968e-06,
          -9.5741e-07, -1.0771e-06]],

        ...,

        [[ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.4361e-06,
          -7.1806e-07, -1.3164e-06],
         [ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.1968e-06,
          -9.5741e-07, -1.0771e-06]],

        [[ 2.1542e-06,  1.1968e-06,  1.4361e-06,  ..., -1.4361e-06,
          

In [18]:
torch.manual_seed(1)

  # size of the minibatch
#z = torch.from_numpy(y)
# load dataset into dataLoader
#print(type(z))
loader = Data.DataLoader(

    dataset=y,

    batch_size=BATCH_SIZE,

    shuffle=True,

    num_workers=0,

)



class DiscreteNet(torch.nn.Module):
        # use an embedding layer to build a NN

    def __init__(self):

        super(DiscreteNet, self).__init__()

        self.embedding = nn.LSTM(37847250,2)
        self.embedding2 = nn.Linear(2,37847250)


    def forward(self, x):

        y,x = self.embedding(x)
        y = self.embedding2(y)
        return y



f = DiscreteNet()


optimizer = optim.Adam(params=f.parameters(), lr=0.001)
loss = nn.MSELoss()


def run():
    for epoch in range(200):

        for step,(batch_x) in enumerate(loader):
            optimizer.zero_grad()
            print(np.shape(batch_x))
            fx = f(batch_x)
            print(np.shape(fx),np.shape(waveform_original))
            l = loss(fx,waveform_original)
            l.backward()
            optimizer.step()
            print("Epoch: ",epoch,"Loss: ",l.item())

    print('Finished Training')


run()

torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.17845749855041504
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.16727745532989502
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.16764624416828156
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.16647100448608398
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.1653381735086441
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.16573187708854675
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.1645735502243042
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  0 Loss:  0.1663

Epoch:  6 Loss:  0.13809636235237122
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  6 Loss:  0.1376800537109375
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  6 Loss:  0.13714435696601868
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  6 Loss:  0.1365409940481186
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  6 Loss:  0.13645923137664795
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  6 Loss:  0.13612881302833557
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  7 Loss:  0.1352917104959488
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  7 Loss:  0.1350603550672531
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2,

torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  12 Loss:  0.11288570612668991
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11244916915893555
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11240541189908981
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11186085641384125
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11145529896020889
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11116350442171097
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11087041348218918
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  13 Loss:  0.11047331243753433
torch.

Epoch:  19 Loss:  0.09244336187839508
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.0920519158244133
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.09176144003868103
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.09147107601165771
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.09127938002347946
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.09089688211679459
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  19 Loss:  0.09066222608089447
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  20 Loss:  0.09034987539052963
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.S

torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  25 Loss:  0.07503372430801392
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  25 Loss:  0.07476944476366043
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Loss:  0.07452265173196793
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Loss:  0.07427285611629486
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Loss:  0.07403263449668884
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Loss:  0.07379356771707535
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Loss:  0.07355453819036484
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  26 Los

...

torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  171 Loss:  0.00017729098908603191
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  171 Loss:  0.0001776633580448106
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  171 Loss:  0.00017553346697241068
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  171 Loss:  0.00017541958368383348
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  171 Loss:  0.00017365196254104376
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  172 Loss:  0.0001731888041831553
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  172 Loss:  0.00017148190818261355
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  172 Loss: 

Epoch:  177 Loss:  0.00012809837062377483
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  177 Loss:  0.00012740734382532537
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  177 Loss:  0.00012708184658549726
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  178 Loss:  0.00012709452130366117
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  178 Loss:  0.0001267701736651361
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  178 Loss:  0.00012450046779122204
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  178 Loss:  0.0001257722033187747
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  178 Loss:  0.0001238269469467923
torch.Size([10, 2, 37847250])
torch.S

torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  9.181044879369438e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  9.239223436452448e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  9.190832497552037e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  9.114389831665903e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  8.990974311018363e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  8.992192306322977e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  184 Loss:  8.970264025265351e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 3784

Epoch:  190 Loss:  6.662402302026749e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.606207170989364e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.548868987010792e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.492279499070719e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.459347787313163e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.505633791675791e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.338650564430282e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  190 Loss:  6.395168747985736e-05
torch.Size([10, 2, 37847250])
torch.Size([

torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  196 Loss:  4.682902726926841e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  196 Loss:  4.6886671043466777e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  196 Loss:  4.6618519263574854e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  196 Loss:  4.588199590216391e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  196 Loss:  4.5815853809472173e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  197 Loss:  4.586579234455712e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  197 Loss:  4.516591798164882e-05
torch.Size([10, 2, 37847250])
torch.Size([10, 2, 37847250]) torch.Size([10, 2, 37847250])
Epoch:  197 Loss:  4

In [21]:
test_audio = torch.tensor(test_audio)
d = f(test_audio.reshape(1,2,37847250))
d = d.reshape(2, 37847250)

wavfile.write('reconstructed_B_full_blank.wav',sample_rate_original,np.transpose(d.detach().numpy()))



C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [23]:
test_audio_1,_ = soundfile.read('data\\2_sets\\A\\'+'full_B_n1.wav') 
test_audio_1 = test_audio_1.astype('float32')
test_audio_1 /=255
test_audio_1 = torch.tensor(test_audio_1)
n = f(test_audio_1.reshape(1,2,37847250))
n = n.reshape(2, 37847250)


wavfile.write('reconstructed_B_full_noisy.wav',sample_rate_original,np.transpose(n.detach().numpy()))



In [24]:
from sklearn.metrics import mean_squared_error 

waveform_original_B,sample_rate_original= soundfile.read('data\\2_sets\\B\\'+'full_B.wav')
waveform_original_B = waveform_original_B.astype('float32')
waveform_original_B /= 255
waveform_original_B = np.repeat(torch.tensor(np.transpose(waveform_original_B))[np.newaxis,...], BATCH_SIZE, axis=0)
# Calculation of Mean Squared Error (MSE) 
mean_squared_error(waveform_original_B[0].detach().numpy(),d.detach().numpy()) 


0.00018203682